In [1]:
from nuq import NuqClassifier

import numpy as np
from tqdm.auto import tqdm

%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'matplotlib'

In [2]:
n_points = 100
circle_r = 10
dim = 2
h = 4

method = 'all_data' # all_data
kernel = 'RBF'
use_uniform_prior = False

In [3]:
def generate_points_on_circle(circle_r=10, circle_x=0, circle_y=0, n_points=10, uniform=True):
    if not uniform:
        alpha = 2 * np.pi * np.random.rand(n_points)
    else:
        alpha = 2 * np.pi * np.linspace(start=0, stop=1, num=n_points)

    x = circle_r * np.cos(alpha) + circle_x
    y = circle_r * np.sin(alpha) + circle_y

    return np.vstack((x, y)).T

In [4]:
X = generate_points_on_circle(n_points=n_points, circle_r=circle_r)

In [5]:
import matplotlib.pyplot as plt
%matplotlib widget

In [6]:
plt.close()
plt.figure()

plt.scatter(X[:, 0], X[:, 1])
plt.scatter(0, 0, color='red')
plt.axis('equal')
plt.show();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
nuq = NuqClassifier(
    kernel_type=kernel,
    use_uniform_prior=use_uniform_prior,
    method=method,
    tune_bandwidth=False,
    bandwidth=np.array([h, h])
)

In [8]:
y = np.zeros(X.shape[0], dtype='int')

In [9]:
nuq.fit(X, y)

NuqClassifier(bandwidth=array([4, 4]), method='all_data', n_neighbors=100,
              tune_bandwidth=False, use_uniform_prior=False)

In [10]:
def test_circle(nuq_trained, dim, r, i=0):
    origin = np.zeros((1, dim))
    rand_vect = np.random.randn(1, dim)
    N = nuq_trained.n_neighbors
    
    kde_pred = nuq_trained.get_kde(origin)
    kernel_output = nuq_trained.kernel(rand_vect, rand_vect)
    uncertainties_pred = nuq_trained.predict_uncertainty(np.zeros((1, 2)), )
    
    aleatoric_pred = uncertainties_pred['aleatoric']
    epistemic_pred = uncertainties_pred['epistemic']
    
    if len(np.unique(nuq_trained.bandwidth)) > 1:
        raise ValueError('Bandwidth must be isotropic')
    h = nuq_trained.bandwidth[0]
    KDE_error = np.abs(-dim * (np.log(2 * np.pi) * 0.5 + np.log(h)) - (r**2 / (2 * h**2)) - kde_pred)
    Kernel_error = np.abs(-dim / 2. * np.log(2 * np.pi) - kernel_output)
    Aleatoric_error = np.abs(np.log(nuq_trained.coeff + min(1 - i / N, i / N)) - aleatoric_pred)
    Epistemic_error = np.abs((6 - dim) * 0.25 * np.log(2.) - 0.5 * np.log(np.pi) - 0.5 * np.log(nuq_trained.n_neighbors) + 
                             0.5 * np.log(nuq_trained.coeff + i * (N - i) / N ** 2) + (r**2 / (4 * h**2)) - epistemic_pred)
    
    
    print(f'{KDE_error=}')
    print(f'{Kernel_error=}')
    print(f'{Aleatoric_error=}')
    print(f'{Epistemic_error=}')
    
    return {
        "kde_pred": kde_pred,
        "kernel_output": kernel_output,
        **uncertainties_pred
    }

In [11]:
# print("KDE error:", -dim * (np.log(2 * np.pi) * 0.5 + np.log(h)) - (circle_r**2 / (2 * h**2)) - nuq.get_kde(np.zeros((1, 2))))

In [12]:
# print("Ue error:", (6 - dim) * 0.25 * np.log(2.) - 0.5 * np.log(np.pi) - 0.5 * np.log(nuq.n_neighbors) + 0.5 * np.log(nuq.coeff) + (circle_r**2 / (4 * h**2))- uncertainties['epistemic'])

In [13]:
# rand_vect = np.random.randn(1, 2)
# print("Kernel:", -dim / 2. * np.log(2 * np.pi) - nuq.kernel(rand_vect, rand_vect))

In [15]:
_ = test_circle(nuq_trained=nuq, dim=dim, r=circle_r)

> /home/nkotelevskii/github/nw_uncertainty/nuq/method/nuq.py(152)predict_uncertainty()
    150                 import pdb
    151                 pdb.set_trace()
--> 152                 Ue = log_half_gaussian_mean(asymptotic_var=log_as_var).squeeze()
    153                 Ua = np.min(f1_hat_y_x, axis=1, keepdims=True)
    154                 if not self.use_uniform_prior:



ipdb>  log_half_gaussian_mean(asymptotic_var=log_as_var)


array([[-6.37576559]])


ipdb>  np.min(f1_hat_y_x, axis=1, keepdims=True)


array([[-inf]])


...

array([[-11.51292546]])


ipdb>  logsumexp(np.concatenate([Ua[None], np.log(self.coeff) * np.ones(shape=broadcast_shape)], axis=0), axis=0).squeeze()


array(-11.51292546)


ipdb>  q


BdbQuit: 

In [ ]:
indices = np.random.permutation(np.arange(n_points))

In [ ]:
y = np.zeros(X.shape[0], dtype='int')

dict_of_results = {
    'epistemic': [],
    'aleatoric': [],
    'total': [],
    'KDE': [],
    'kernel': []
}

for i in tqdm(range(n_points + 1)):
    
    if i > 0:
        ind = indices[i - 1]
        y[ind] = 1
    
    nuq = NuqClassifier(
        method=method,
        tune_bandwidth=False,
        bandwidth=np.array([h, h])
    )
    
    nuq.fit(X, y)
    res = test_circle(nuq_trained=nuq, dim=dim, r=circle_r, i=i)
    
    dict_of_results['epistemic'].append(res['epistemic'].squeeze())
    dict_of_results['aleatoric'].append(res['aleatoric'].squeeze())
    dict_of_results['total'].append(res['total'].squeeze())
    dict_of_results['KDE'].append(res['kde_pred'].squeeze())
    dict_of_results['kernel'].append(res['kernel_output'].squeeze())

In [ ]:
plt.close()
_, ax = plt.subplots(nrows=1, ncols=len(dict_of_results.keys()), sharex=True, figsize=(12, 4))

for i, key in enumerate(dict_of_results.keys()):
    ax[i].set_title(key)
    ax[i].plot(np.arange(n_points + 1), dict_of_results[key])

plt.show();

In [ ]:
y = np.zeros(X.shape[0], dtype='int')

dict_of_results = {
    'epistemic': [],
    'aleatoric': [],
    'total': [],
    'KDE': [],
    'kernel': []
}

radii = [1, 5, 10, 20, 50, 100]

for r in tqdm(radii):
    
    X = generate_points_on_circle(n_points=n_points, circle_r=r)
    
    nuq = NuqClassifier(
        method=method,
        tune_bandwidth=False,
        bandwidth=np.array([h, h])
    )
    
    nuq.fit(X, y)
    res = test_circle(nuq_trained=nuq, dim=dim, r=r, i=0)
    
    dict_of_results['epistemic'].append(res['epistemic'].squeeze())
    dict_of_results['aleatoric'].append(res['aleatoric'].squeeze())
    dict_of_results['total'].append(res['total'].squeeze())
    dict_of_results['KDE'].append(res['kde_pred'].squeeze())
    dict_of_results['kernel'].append(res['kernel_output'].squeeze())

In [ ]:
plt.close()
_, ax = plt.subplots(nrows=1, ncols=len(dict_of_results.keys()), sharex=True, figsize=(12, 4))

for i, key in enumerate(dict_of_results.keys()):
    ax[i].set_title(key)
    ax[i].plot(radii, dict_of_results[key])

plt.show();